In [ ]:
#！/usr/bin/python
# -*- coding: utf-8 -*-

In [ ]:
import pyodbc
import textwrap
import pandas as pd
import numpy as np

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
sql = textwrap.dedent("""
    SELECT r.userid
          ,h.category
          ,r.website
          ,COUNT(r.url) AS [Visit Numbers]
          ,COUNT(DISTINCT r.date) AS [Visit Days]
          ,DATEDIFF(day ,MAX(r.date), '2017-08-31') AS [Last Visit]
    FROM [data].[dbo].[health_records] r
    JOIN [data].[dbo].[healthsites] h
    ON r.website = h.abbreviation
    GROUP BY r.userid, h.category, r.website
    ORDER BY r.userid, h.category, r.website
""")

In [ ]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=data;')

cursor = cnxn.cursor()

In [ ]:
rows=cursor.execute(sql).fetchall()

In [ ]:
df = pd.DataFrame( [[ij for ij in i] for i in rows] )
df.rename(columns={0:'userid', 1: 'category', 2:'website', 3:'number_of_visits', 4:'visit_frequency', 5:'last_visit'}, inplace=True)
# 计算平均每天的访问量，并四舍五入取整
df['average_daily_visits'] = round(df['number_of_visits']/df['visit_frequency']).apply(int)
df

In [ ]:
plotly.tools.set_credentials_file(username='tenaghirmb', api_key='please_use_your_own_key')

plotly.offline.init_notebook_mode(connected=True)

In [ ]:
# 从原始查询结果中提取作图所需数据
def df_filter(category):
    if category == 'all':
        # 计算每个指标的平均值
        dfx = df.groupby('category').agg(np.mean)
    else:
        dfx = df[df['category']==category]
        dfx = dfx.groupby( 'website').agg(np.mean)   
    # 对逆指标进行处理
    dfx['last_visit'] = dfx['last_visit'].transform(lambda x:1/x)
    # 归一化
    dfx = dfx.transform(lambda x: x / x.max())
    return dfx

In [ ]:
def make_scatterpolar(result,n):
    # 使雷达图的连线闭合，轨迹需回到起点
    l = list(result.iloc[n])
    l.append(l[0])
    return go.Scatterpolar(
      r = l,
      theta = ['Visit Times','Visit Days','Recent Visit Time', 'Average Daily Visit Times', 'Visit Times'],
      #fill = 'toself',
      #opacity = 0.4,
      # 取索引值（category或website的值），groupby后索引不再是数字
      name = result.index.tolist()[n],
      mode = 'lines+markers'
    )

In [ ]:
def make_radar_chart(category='all'):
    dfx = df_filter(category)
    data = []
    for x in range(len(dfx)):
        data.insert(0,make_scatterpolar(dfx,x))
    fig = go.Figure(data=data)
    plotly.offline.iplot(fig, filename = "Loyalty")
    #py.iplot(fig, filename = "Loyalty - "+category)

In [ ]:
make_radar_chart()